## Wave Gage Directional Estimates
This file will compute the directional wave statistics from the wave gages under the bridge.

In [1]:
import numpy as np
import os
from scipy import signal
import sys
# from scipy import pi
import scipy as sp
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd


path = '/Users/cmbaker9/Documents/Research/Lab_Experiments/'
sys.path.append(path + 'codes/pyDIWASP/')
from dirspec import dirspec
# from dirspec import dirspec as dirspec
figpath = path + 'raw/figures/'

In [2]:
Tinfo = {
    "Hs": 0.3,
    "Tp": 2,
    "tide": 1.07,
    "spread": 40,
    "day": '5',
    "trial": '07',
    "insitufile": "08-30-2018-2129UTC"
}
datapath = path + 'data/raw/insitu/Trials/random' + Tinfo['day'] + '/Trial' + Tinfo['trial'] + '/'
xyfile = path + 'data/raw/insitu/wgXY.txt'

In [3]:

wgno = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]
for i in range(len(wgno)):
    wgname = 'wg' + str(wgno[i])
    wgfile = datapath + wgname + '.txt'
    
    f = open(wgfile, "r")
    lines = f.readlines()
    f.close
    x = float(lines[65].split()[2])
    y = float(lines[66].split()[2])
    
    if i == 0: 
        temp=np.loadtxt(wgfile, delimiter=',', comments='%', unpack=True)
        df = pd.DataFrame(temp,columns=['wg1'])
        dxy = pd.DataFrame([x, y],columns=['wg1'])
#         dXY = [x, y]
    else:
        df[wgname] = pd.read_csv(wgfile, delimiter=',', comment='%', header=None)
        dxy[wgname] = [x, y]
#         dXY = dXY.append([x, y])
#         dXY = np.append([x, y])
# !head -n 15 $wgfile
dXY = dxy.values

In [18]:
Options = {'MESSAGE':1, 'PLOTTYPE':1, 'FILEOUT':''}
type(Options)

dict

In [13]:
SM = dict()
EP = dict()
ID = dict()
SM['dirs'] = np.arange(0,360)
SM['freqs'] = np.arange(0.001,3.001,0.001)
SM['S'] = np.zeros([len(SM['freqs']),len(SM['dirs'])])
EP['nfft'] = 1024
EP['dres'] = 360
EP['method'] = 'IMLM'
ID['datatypes'] = ['elev']
for i in range(0,13):
    ID['datatypes'].append('elev')
ID['depth'] = 1.0
ID['data'] = df.values
N = dxy.shape[1]
ID['layout'] = np.concatenate((np.array([np.transpose(dXY[0]), np.transpose(dXY[1])]), np.zeros([1,N])), axis=0)
ID['fs'] = 100
Options_ = []
[Smout,EPout] = dirspec(ID,SM,EP,Options_)


calculating.....

cross power spectra
wavenumbers
transfer parameters



/Users/cmbaker9/Documents/Research/Lab_Experiments/codes/pyDIWASP/private/wavenumber.py:28: RuntimeWarning: overflow encountered in cosh
  ch = np.cosh(a1)


directional spectra using IMLM method
calculating for frequency 1 of 30
calculating for frequency 2 of 30
calculating for frequency 3 of 30
calculating for frequency 4 of 30
calculating for frequency 5 of 30
calculating for frequency 6 of 30
calculating for frequency 7 of 30
calculating for frequency 8 of 30
calculating for frequency 9 of 30
calculating for frequency 10 of 30
calculating for frequency 11 of 30
calculating for frequency 12 of 30
calculating for frequency 13 of 30
calculating for frequency 14 of 30
calculating for frequency 15 of 30
calculating for frequency 16 of 30
calculating for frequency 17 of 30
calculating for frequency 18 of 30
calculating for frequency 19 of 30
calculating for frequency 20 of 30
calculating for frequency 21 of 30
calculating for frequency 22 of 30
calculating for frequency 23 of 30
calculating for frequency 24 of 30
calculating for frequency 25 of 30
calculating for frequency 26 of 30
calculating for frequency 27 of 30
calculating for frequency 

TypeError: can't multiply sequence by non-int of type 'float'

In [7]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.signal import detrend
from interpspec import interpspec
from infospec import infospec
from writespec import writespec
from plotspec import plotspec
from private.velx import velx
from private.vely import vely
from private.pres import pres
from private.elev import elev
from private.wavenumber import wavenumber
from private.IMLM import IMLM
from private.smoothspec import smoothspec
from private.diwasp_csd import diwasp_csd
from private.check_data import check_data


"""
DIWASP V1.4 function
dirspec: main spectral estimation routine

[SMout,EPout]=dirspec(ID,SM,EP,{options})

Outputs:
SMout	    A spectral matrix structure containing the results
Epout		The estimation parameters structure with the values actually used for the computation including any default settings.

Inputs:
ID			An instrument data structure containing the measured data
SM   		A spectral matrix structure; data in field SM.S is ignored.
EP		    The estimation parameters structure. For default values enter EP as []
[options]  options entered as cell array with parameter/value pairs: e.g.{'MESSAGE',1,'PLOTTYPE',2};
                Available options with default values:
                    'MESSAGE',1,    Level of screen display: 0,1,2 (increasing output)
                    'PLOTTYPE',1,   Plot type: 0 none, 1 3d surface, 2 polar type plot, 3 3d surface(compass angles), 4 polar plot(compass angles)
                    'FILEOUT',''  	 Filename for output file: empty string means no file output

Input structures ID and SM are required. Either [EP] or [options] can be included but must be in order if both are included.
"help data_structures" for information on the DIWASP data structures

All of the implemented calculation algorithms are as described by:
Hashimoto,N. 1997 "Analysis of the directional wave spectrum from field data" 
In: Advances in Coastal Engineering Vol.3. Ed:Liu,P.L-F. Pub:World Scientific,Singapore


Original copyright (C) 2002 Coastal Oceanography Group, CWR, UWA, Perth

Translated by Chuan Li and Spicer Bak,
Field Research Facility, US Army Corps of Engineers

Christine M. Baker edited script. The input EP file in form dictionary (line 85)
"""

Options = {'MESSAGE':1, 'PLOTTYPE':1, 'FILEOUT':''}

nopts = len(Options_)

if nopts != 0:
    if nopts % 2 != 0:
        warnings.warn('Options must be in Name/Value pairs - setting to '
            'defaults')
    else:
        for i in range(int(nopts / 2)):
            arg = Options_[2 * i + 1]
            field = Options_[2 * i]
            Options[field] = arg

ptype = Options['PLOTTYPE']; displ = Options['MESSAGE']

print('\ncalculating.....\n\ncross power spectra')

data = detrend(ID['data'], axis=0)
ndat, szd = np.shape(ID['data'])

#get resolution of FFT - if not specified, calculate a sensible value
#if EP['nfft'] == 0:
if 'nfft' in EP:
    nfft = int(EP['nfft'])
else:
    nfft = int(2 ** (8 + np.round(np.log2(ID['fs']))))
    EP['nfft'] = nfft
if nfft > ndat: raise Exception('Data length of {} too small'.format(dat))

#calculate the cross-power spectra
xps = np.empty((szd, szd, int(nfft / 2)), 'complex128')
for m in range(szd):
    for n in range(szd):
        xpstmp, Ftmp = diwasp_csd(data[:, m], data[:, n], nfft, ID['fs'])
        xps[m, n, :] = xpstmp[1:int(nfft / 2) + 1]
F = Ftmp[1:int(nfft / 2) + 1]; nf = int(nfft / 2)

print('wavenumbers')
wns = wavenumber(2  * np.pi * F, ID['depth'] * np.ones(np.shape(F)))
pidirs = np.linspace(-np.pi, np.pi - 2 * np.pi / EP['dres'], num=int(2 * np.pi / (2 * np.pi / EP['dres']))) # CB changed

#calculate transfer parameters
print('transfer parameters\n')
trm = np.empty((szd, nf, len(pidirs)))
kx = np.empty((szd, szd, nf, len(pidirs)))
for m in range(szd):
    trm[m, :, :] = eval(ID['datatypes'][m])(2 * np.pi * F, pidirs, wns, 
        ID['layout'][2, m], ID['depth'])
    for n in range(szd):
        kx[m, n, :, :] = wns[:, np.newaxis] * ((ID['layout'][0, n] - 
            ID['layout'][0, m]) * np.cos(pidirs) + (ID['layout'][1, n] - 
            ID['layout'][1, m]) * np.sin(pidirs))

Ss = np.empty((szd, nf), dtype='complex128')
for m in range(szd):
    tfn = trm[m, :, :]
    Sxps = xps[m, m, :]
    Ss[m, :] = Sxps / (np.max(tfn, axis=1) * np.conj(np.max(tfn, axis=1)))

ffs = np.logical_and(F >= np.min(SM['freqs']), F <= np.max(SM['freqs']))
SM1 = dict()
SM1['freqs'] = F[ffs]
SM1['funit'] = 'Hz'
SM1['dirs'] = pidirs
SM1['dunit'] = 'rad'

# call appropriate estimation function
print('directional spectra using {} method'.format(EP['method']))
SM1['S'] = eval(EP['method'])(xps[:, :, ffs], trm[:, ffs, :], 
    kx[:, :, ffs, :], Ss[:, ffs], pidirs, EP['iter'], displ)
SM1['S'][np.logical_or(np.isnan(SM1['S']), SM1['S'] < 0)] = 0

#Interpolate onto user specified matrix
print('\ninterpolating onto specified matrix...\n')
# SMout = interpspec(SM1, SM)


calculating.....

cross power spectra
wavenumbers
transfer parameters



/Users/cmbaker9/Documents/Research/Lab_Experiments/codes/pyDIWASP/private/wavenumber.py:28: RuntimeWarning: overflow encountered in cosh
  ch = np.cosh(a1)


directional spectra using IMLM method
calculating for frequency 1 of 30
calculating for frequency 2 of 30
calculating for frequency 3 of 30
calculating for frequency 4 of 30
calculating for frequency 5 of 30
calculating for frequency 6 of 30
calculating for frequency 7 of 30
calculating for frequency 8 of 30
calculating for frequency 9 of 30
calculating for frequency 10 of 30
calculating for frequency 11 of 30
calculating for frequency 12 of 30
calculating for frequency 13 of 30
calculating for frequency 14 of 30
calculating for frequency 15 of 30
calculating for frequency 16 of 30
calculating for frequency 17 of 30
calculating for frequency 18 of 30
calculating for frequency 19 of 30
calculating for frequency 20 of 30
calculating for frequency 21 of 30
calculating for frequency 22 of 30
calculating for frequency 23 of 30
calculating for frequency 24 of 30
calculating for frequency 25 of 30
calculating for frequency 26 of 30
calculating for frequency 27 of 30
calculating for frequency 

In [11]:
import numpy as np
from warnings import warn
from scipy.interpolate import griddata
from private.hsig import hsig
from private.spectobasis import spectobasis

SMin = SM1
SMout = SM

Hs1 = hsig(SMin)
    
SMin, facin = spectobasis(SMin)
SMtmp, facout = spectobasis(SMout)

s1 = SMin['freqs'][:, np.newaxis] * np.sin(SMin['dirs'])
c1 = SMin['freqs'][:, np.newaxis] * np.cos(SMin['dirs'])
s2 = SMtmp['freqs'][:, np.newaxis] * np.sin(SMtmp['dirs'])
c2 = SMtmp['freqs'][:, np.newaxis] * np.cos(SMtmp['dirs'])

Stmp = griddata((s1.flatten(), c1.flatten()), SMin['S'].flatten(), 
    (s2.flatten(), c2.flatten())).reshape(s2.shape)
Stmp[np.isnan(Stmp)] = 0
SMout['S'] = Stmp / facout

# check Hsig of mapped spectrum and check sufficiently close to original
Hs2 = hsig(SMout)
if (Hs2 - Hs1) / Hs1 > 0.02:
    warn('User defined grid may be too coarse; try increasing' +
        ' resolution of ''SM[\'freqs\']'' or ''SM[\'dirs\']''')

TypeError: can't multiply sequence by non-int of type 'float'